In [3]:
## Step 2: Backtest framework

# You will generate a signal:

#1 = long
#0 = cash
#The backtest uses position = signal.shift(1) to avoid look-ahead bias.

# Define the function which will develop


## Step 3: Implement one strategy

# Implement **ONE** of the following by producing a `signal` series (0/1). Comment out the options you are not using.

# **Guidance:** Choose which strategy option (A1, A2, or A3) you will implement and use only that signal. For whichever strategy you pick, decide on appropriate parameter values:
#- If using **ROC Threshold**: select the lookback window `roc_n` and a percentage threshold. (e.g., `roc_n=10` and `roc_threshold=0.03` means go long if price has risen more than 3% over the last 10 days.)
#- If using **MA Crossover**: choose short (`fast_n`) and long (`slow_n`) moving average lengths. (A typical choice might be 20-day vs 50-day MAs, but you can experiment.)
#- If using **MACD**: you can use the standard periods (12, 26, 9) for fast EMA, slow EMA, and signal line, or other values if justified.

#> **Checkpoint 2:** After developing one trading strategy, considering committing your notebook (e.g., "Completed ROC Threshold").

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Load Data
# Download price data for AAPL
ticker = "AAPL"
start_date = "2021-01-01"
end_date = "2026-01-01"

price_data = yf.download(ticker, start_date, end_date)
price_data = price_data[['Close']]

print(price_data.head().round(2))

[*********************100%***********************]  1 of 1 completed

Price        Close
Ticker        AAPL
Date              
2021-01-04  125.97
2021-01-05  127.53
2021-01-06  123.24
2021-01-07  127.44
2021-01-08  128.54


In [5]:
## STEP 2: The Backtest Framework

# Create a column for signals and fill with 0 (In cash default)
# Creating the container for our strategy, for when we calculate ROC in Step3 
price_data['signal'] = 0

# Create column for position
# This creates a NEW column named position, taking the signal column and moving everything down 1 row
price_data['position'] = price_data['signal'].shift(1)

#Prevents 'look-ahead bias'
# The quotation marks means Python treats it as a label (string of text) Without them, python thinks it is a variable. 

In [12]:

#1. Creating Parameters
roc_n = 10
roc_threshold = 0.03

#2.. Calculating Rate of Change
price_data['ROC'] = price_data['Close'].pct_change(periods=roc_n)

#3. Generating the signal (The Decision)
# 1 if it is above 3%, 0 if it is not
price_data.loc[price_data['ROC'] > roc_threshold, 'signal'] = 1

#4. Updating Position (The Backtest Rule)
#Shift signal by 1 day to ensure we trade on the next day's open
price_data['position'] = price_data['signal'].shift(1)

#5. Clean up data
# First 10 rows are empty due to lookback period
price_data = price_data.dropna()

#6. Display data to confirm it worked
long_days = price_data[price_data['signal'] == 1]
cash_days = price_data [price_data['signal'] == 0]

print(f"Analysis for: {ticker}")
print(f"Lookback: Threshold of {roc_threshold*100}% over {roc_n} days")
print(f"First valid trading date: {price_data.index[0].date()}")
print(f"Number of signals found: {len(long_days)}")

# 6. Final Validation
display(price_data[['Close', 'ROC', 'signal', 'position']].head())
display(price_data[['Close', 'ROC', 'signal', 'position']].tail(10))

display(cash_days[['Close', 'ROC', 'signal', 'position']].head())
#Proof of the backtest logic - demonstrates strategy adheres to 'next-day' trading rule and handles closures over the weekend. Avoids look-ahead bias and functions accurately within real-world market constraints. 

Analysis for: AAPL
Lookback: Threshold of 3.0% over 10 days
First valid trading date: 2021-01-27
Number of signals found: 413


Price,Close,ROC,signal,position
Ticker,AAPL,,,
Date,,,,
2021-01-27,138.288620,0.102950,1,1.0
2021-01-28,133.450577,0.047368,1,1.0
2021-01-29,128.456802,0.023660,0,1.0
2021-02-01,130.578903,0.055057,1,0.0
2021-02-02,131.406326,0.056012,1,1.0


Price,Close,ROC,signal,position
Ticker,AAPL,,,
Date,,,,
2025-12-17,271.839996,-0.043322,0,0.0
2025-12-18,272.190002,-0.030317,0,0.0
2025-12-19,273.670013,-0.018330,0,0.0
2025-12-22,270.970001,-0.024902,0,0.0
2025-12-23,272.359985,-0.017389,0,0.0
2025-12-24,273.809998,-0.017828,0,0.0
2025-12-26,273.399994,-0.016653,0,0.0
2025-12-29,273.760010,-0.016243,0,0.0


Price,Close,ROC,signal,position
Ticker,AAPL,,,
Date,,,,
2021-01-29,128.456802,0.023660,0,1.0
2021-02-03,130.384201,0.014467,0,1.0
2021-02-04,133.742584,0.003799,0,0.0
2021-02-05,133.328278,-0.015141,0,0.0
2021-02-08,133.474503,-0.040620,0,0.0
